# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_next_token_predictions(prompt_text: str, model_name: str = "gpt2", top_k: int = 10):
    """
    指定されたプロンプトに続く次のトークンの上位K個とその確率を予測します。
    また、プロンプトがどのようにトークン化されたかを表示します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。
                                     デフォルトは "gpt2"。
        top_k (int, optional): 取得する上位トークンの数。デフォルトは 10。

    Returns:
        list: 予測されたトークンとその確率のリスト。
              各要素は (トークン文字列, 確率) のタプル。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # モデルを評価モードに設定 (勾配計算をオフにするなど)
        model.eval()

        # 2. プロンプトのトークン化
        print(f"\n--- Tokenizing Prompt: \"{prompt_text}\" ---")
        # トークナイザーによっては、プロンプトの先頭にスペースが必要な場合があるため、
        # GPT-2のようなモデルではそのままで良いが、モデルに応じて調整することも考慮
        inputs = tokenizer(prompt_text, return_tensors="pt")
        input_ids = inputs["input_ids"]

        print(f"Input IDs: {input_ids.tolist()[0]}")
        tokens = [tokenizer.decode([token_id]) for token_id in input_ids[0].tolist()]
        # トークンによっては、デコード時に不要なスペースが含まれることがあるため、必要に応じて調整
        # 例: tokens = [tokenizer.convert_ids_to_tokens(token_id) for token_id in input_ids[0].tolist()]
        print(f"Tokens: {tokens}")
        print("--------------------------------------")

        # 3. 次のトークンの予測
        with torch.no_grad(): # 勾配計算を行わないコンテキスト
            outputs = model(input_ids)
            # outputs.logits の形状は (batch_size, sequence_length, vocab_size)
            # 次のトークンの予測なので、最後のトークン位置のlogitsを使用
            next_token_logits = outputs.logits[0, -1, :]

        # 4. Logitsを確率に変換 (Softmax)
        probabilities = torch.softmax(next_token_logits, dim=-1)

        # 5. 上位K個のトークンとその確率を取得
        top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k)

        results = []
        print(f"\n--- Top {top_k} next token predictions for \"{prompt_text}\" ---")
        for i in range(top_k):
            token_id = top_k_indices[i].item()
            token_probability = top_k_probabilities[i].item()
            # token_idを文字列にデコード
            # decodeメソッドはリスト形式のIDも受け付ける
            token_string = tokenizer.decode([token_id])
            results.append((token_string, token_probability))
            print(f"{i+1}. Token: \"{token_string}\" (ID: {token_id}), Probability: {token_probability:.6f}")
        print("----------------------------------------------------")
        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

if __name__ == "__main__":
    prompt = "The movie was full of"
    # GPT-2モデルを使用します。他のモデル (例: "gpt2-medium", "gpt2-large", "EleutherAI/gpt-neo-1.3B" など) も試せます。
    # 大きなモデルほど、より高品質な予測が期待できますが、計算リソースも多く必要とします。
    predicted_tokens = get_next_token_predictions(prompt, model_name="gpt2", top_k=10)

    # (オプション) 他のGPT系モデルを試す場合
    # print("\nTrying with gpt2-medium (may take longer to download/run):")
    # predicted_tokens_medium = get_next_token_predictions(prompt, model_name="gpt2-medium", top_k=10)

Loading model and tokenizer for 'gpt2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


--- Tokenizing Prompt: "The movie was full of" ---
Input IDs: [464, 3807, 373, 1336, 286]
Tokens: ['The', ' movie', ' was', ' full', ' of']
--------------------------------------

--- Top 10 next token predictions for "The movie was full of" ---
1. Token: " jokes" (ID: 14532), Probability: 0.021892
2. Token: " great" (ID: 1049), Probability: 0.018644
3. Token: " laughs" (ID: 22051), Probability: 0.011524
4. Token: " bad" (ID: 2089), Probability: 0.010874
5. Token: " surprises" (ID: 24072), Probability: 0.010667
6. Token: " references" (ID: 10288), Probability: 0.010528
7. Token: " fun" (ID: 1257), Probability: 0.009992
8. Token: " humor" (ID: 14733), Probability: 0.007415
9. Token: " "" (ID: 366), Probability: 0.007408
10. Token: " the" (ID: 262), Probability: 0.006709
----------------------------------------------------


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text_with_varying_strategies(prompt_text: str, model_name: str = "gpt2"):
    """
    指定されたプロンプトに対し、異なるデコーディング戦略やパラメータを用いて
    複数のテキストを生成し、その結果を表示します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。
                                     デフォルトは "gpt2"。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # トークナイザーにパディングトークンが設定されていない場合、EOSトークンを設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id


        # モデルを評価モードに設定
        model.eval()

        # プロンプトをトークンIDに変換
        # attention_maskも一緒に渡すことで、パディングトークンを無視できる
        inputs = tokenizer(prompt_text, return_tensors="pt", padding=True)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]


        # 生成するテキストの共通パラメータ
        # max_lengthはプロンプトの長さを考慮して設定。ここではプロンプトに加えて約30トークン生成。
        # プロンプトのトークン数を取得
        prompt_num_tokens = input_ids.shape[-1]
        max_new_tokens = 30 # 新しく生成するトークンの最大数
        max_length = prompt_num_tokens + max_new_tokens

        num_return_sequences = 2 # 各設定で生成するシーケンスの数

        print(f"\n--- Prompt: \"{prompt_text}\" ---")
        print(f"(Generating up to {max_new_tokens} new tokens beyond the prompt)")

        # --- 1. Greedy Search ---
        # 各ステップで最も確率の高いトークンを選択。決定的で高速だが、単調になりやすい。
        print("\n--- Strategy 1: Greedy Search ---")
        greedy_outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1, # Greedy Searchは通常1つのシーケンスを返す
            do_sample=False # サンプリングを無効化
        )
        for i, output in enumerate(greedy_outputs):
            decoded_output = tokenizer.decode(output, skip_special_tokens=True)
            print(f"Greedy Output {i+1}: {decoded_output}")

        # --- 2. Beam Search ---
        # 複数の候補（ビーム）を保持し、全体として最も良いシーケンスを探す。
        # Greedyより高品質な傾向があるが、計算コストが高く、多様性に欠けることも。
        num_beams = 5
        print(f"\n--- Strategy 2: Beam Search (num_beams={num_beams}) ---")
        beam_outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            early_stopping=True, # 全てのビームがEOSに到達したら停止
            no_repeat_ngram_size=2 # 同じn-gramの繰り返しを避ける (例: "I think I think")
        )
        for i, output in enumerate(beam_outputs):
            decoded_output = tokenizer.decode(output, skip_special_tokens=True)
            print(f"Beam Output {i+1} (num_beams={num_beams}): {decoded_output}")

        # --- 3. Sampling with varying Temperature ---
        # 次のトークンを確率分布に基づいてランダムに選択。
        # Temperature: 値が低いと高確率のトークンが選ばれやすく（決定的）、高いと低確率のトークンも選ばれ（ランダム性が増す）。
        temperatures = [0.7, 1.0, 1.5]
        print("\n--- Strategy 3: Sampling with varying Temperature (top_k=50) ---")
        for temp in temperatures:
            print(f"  - Temperature = {temp}")
            sampling_outputs_temp = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True, # サンプリングを有効化
                temperature=temp,
                top_k=50, # 上位50件のトークンからサンプリング
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_temp):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")

        # --- 4. Sampling with Top-K ---
        # Top-K Sampling: 確率上位K個のトークンのみをサンプリング対象とする。
        top_k_values = [10, 50] # top_k=0で無効化 (通常のtemperature samplingになる)
        print(f"\n--- Strategy 4: Sampling with Top-K (Temperature=1.0) ---")
        for k_val in top_k_values:
            print(f"  - Top-K = {k_val}")
            sampling_outputs_top_k = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True,
                temperature=1.0, # Temperatureは固定してTop-Kの効果を見る
                top_k=k_val,
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_top_k):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")

        # --- 5. Sampling with Top-P (Nucleus Sampling) ---
        # Top-P Sampling: 確率の累積がPを超えるまでの最小のトークンセットをサンプリング対象とする。
        # 候補の数を動的に調整する。
        top_p_values = [0.90, 0.95] # top_p=1.0で無効化
        print(f"\n--- Strategy 5: Sampling with Top-P (Nucleus Sampling, Temperature=1.0) ---")
        for p_val in top_p_values:
            print(f"  - Top-P = {p_val}")
            # top_k=0 を設定して、top_pサンプリングを確実に有効にする
            sampling_outputs_top_p = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True,
                temperature=1.0,
                top_p=p_val,
                top_k=0, # Top-KとTop-Pは通常どちらかを主に使用するため、Top-Kは無効化
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_top_p):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")
        print("----------------------------------------------------")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    prompt = "The movie was full of"
    # "gpt2" は比較的小さなモデルです。より大きなモデル (例: "gpt2-medium", "EleutherAI/gpt-neo-1.3B") も試せますが、
    # リソースと時間が必要になります。
    generate_text_with_varying_strategies(prompt, model_name="gpt2")

Loading model and tokenizer for 'gpt2'...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Prompt: "The movie was full of" ---
(Generating up to 30 new tokens beyond the prompt)

--- Strategy 1: Greedy Search ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Greedy Output 1: The movie was full of jokes and jokes about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the

--- Strategy 2: Beam Search (num_beams=5) ---


...

    Output 1: The movie was full of the kinds of people who were really excited to see it, including myself. I was like, 'Holy shit, this movie is really cool.' And
    Output 2: The movie was full of humor, and it was a great way to show off the characters. It made a lot of people laugh.

What did you think about how
  - Top-K = 50


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


    Output 1: The movie was full of heartache, and many of its key characters, namely Shizune and Yuki, turned on their own.

According to sources close to
    Output 2: The movie was full of many jokes because it reminded me of those with a certain feel about them. They're all in the same movie, and that's what it really comes

--- Strategy 5: Sampling with Top-P (Nucleus Sampling, Temperature=1.0) ---
  - Top-P = 0.9


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


    Output 1: The movie was full of uncomfortable references to "SNL," with long pauses to swap out to Pizarro, a recurring character in the original series who died a few years
    Output 2: The movie was full of talk about "lethal weapons." But similar characters in a very real movie, with some notable differences, appear more frequently in serials, notably these world
  - Top-P = 0.95
    Output 1: The movie was full of unexpected twists and has been starred in a host of other blockbuster films.

But some of the actors' star power stems from the chemistry between the
    Output 2: The movie was full of black humour to begin with and kept going. As you have seen in the story I am a fan of Blade Runner but these are lesser known movies that
----------------------------------------------------


## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [3]:
import torch
import torch.nn.functional as F # Softmax関数を利用するためにインポート
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text_with_token_probabilities(prompt_text: str, model_name: str = "gpt2", max_new_tokens: int = 10):
    """
    指定されたプロンプトに続くテキストを生成し、各生成トークンの尤度（確率）を表示します。
    Greedy Searchを使用します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。デフォルトは "gpt2"。
        max_new_tokens (int, optional): 新しく生成するトークンの最大数。デフォルトは 10。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # モデルを評価モードに設定
        model.eval()

        # パディングトークンが未設定の場合、EOSトークンをパディングトークンとして設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        # 2. プロンプトのトークン化
        inputs = tokenizer(prompt_text, return_tensors="pt")
        input_ids = inputs["input_ids"]
        # attention_mask は generate メソッド内で input_ids に基づいて自動生成されることが多いですが、
        # 明示的に渡すことも可能です。ここでは省略し、generateメソッドのデフォルトの振る舞いに任せます。

        print(f"\n--- Prompt: \"{prompt_text}\" ---")

        # 3. テキスト生成とスコアの取得
        # output_scores=True: 各生成ステップでの全語彙に対するスコア(logits)を返す
        # return_dict_in_generate=True: 出力を辞書形式で受け取り、'sequences'や'scores'にアクセスしやすくする
        # do_sample=False: Greedy Searchを行う (最も確率の高いトークンを選択)
        with torch.no_grad(): # 勾配計算を無効化
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                output_scores=True,
                return_dict_in_generate=True,
                do_sample=False # Greedy search
                # --- サンプリングで試す場合の例 ---
                # do_sample=True,
                # temperature=0.7,
                # top_k=50,
            )

        # 生成されたシーケンス全体 (プロンプト部分も含む)
        # outputs.sequences の形状は (batch_size, sequence_length)
        # ここでは batch_size = 1 を想定
        generated_sequence_ids = outputs.sequences[0]

        # 各生成ステップでのロジットのタプル
        # outputs.scores の各要素は、そのステップで生成されるトークンの全語彙に対するロジット
        # タプルの長さは max_new_tokens と同じ
        # 各ロジットテンソルの形状は (batch_size, vocab_size)
        step_scores = outputs.scores

        # プロンプト部分の長さを取得
        prompt_length = input_ids.shape[1]

        print(f"\n--- Generated Text with Token Probabilities (Strategy: Greedy Search, Max New Tokens: {max_new_tokens}) ---")

        generated_tokens_with_probs = []

        for i in range(len(step_scores)): # 生成された各ステップについてループ
            # i番目のステップで生成されたトークンのID (プロンプトの次から数えてi番目)
            # generated_sequence_ids の (prompt_length + i) 番目の要素
            actual_generated_token_id = generated_sequence_ids[prompt_length + i].item()

            # i番目のステップのロジットを取得 (batch_size=1を想定)
            # step_scores[i] の形状は (1, vocab_size)
            logits_for_this_step = step_scores[i][0] # batch_size次元を削除

            # ロジットを確率に変換
            probabilities_for_this_step = F.softmax(logits_for_this_step, dim=-1)

            # 実際に生成されたトークンの確率を取得
            probability_of_actual_token = probabilities_for_this_step[actual_generated_token_id].item()

            # トークンIDを文字列にデコード
            token_string = tokenizer.decode([actual_generated_token_id])

            generated_tokens_with_probs.append({
                "token_str": token_string,
                "token_id": actual_generated_token_id,
                "probability": probability_of_actual_token
            })
            print(f"Step {i+1}: Token: \"{token_string}\" (ID: {actual_generated_token_id}), Probability: {probability_of_actual_token:.4f}")

        # 全体の生成テキストを表示
        full_generated_text = tokenizer.decode(generated_sequence_ids, skip_special_tokens=True)
        print(f"\nFull generated text: {full_generated_text}")
        print("----------------------------------------------------")

        return generated_tokens_with_probs

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
        return []

if __name__ == "__main__":
    prompt = "The movie was full of"
    # 新しく生成するトークン数を10に設定
    generated_info = generate_text_with_token_probabilities(prompt, model_name="gpt2", max_new_tokens=10)

    # (オプション) サンプリング戦略で試す場合
    # その場合は、generate_text_with_token_probabilities 関数の model.generate の呼び出し部分で
    # do_sample=True とし、必要に応じて temperature, top_k, top_p などのパラメータを設定してください。
    # サンプリングの場合、表示される確率は「そのステップでサンプリングによって選ばれたトークンが元々持っていた確率」となります。

Loading model and tokenizer for 'gpt2'...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Prompt: "The movie was full of" ---

--- Generated Text with Token Probabilities (Strategy: Greedy Search, Max New Tokens: 10) ---
Step 1: Token: " jokes" (ID: 14532), Probability: 0.0219
Step 2: Token: " and" (ID: 290), Probability: 0.2892
Step 3: Token: " jokes" (ID: 14532), Probability: 0.0985
Step 4: Token: " about" (ID: 546), Probability: 0.2056
Step 5: Token: " how" (ID: 703), Probability: 0.0997
Step 6: Token: " the" (ID: 262), Probability: 0.0846
Step 7: Token: " movie" (ID: 3807), Probability: 0.0364
Step 8: Token: " was" (ID: 373), Probability: 0.2963
Step 9: Token: " a" (ID: 257), Probability: 0.0677
Step 10: Token: " joke" (ID: 9707), Probability: 0.1735

Full generated text: The movie was full of jokes and jokes about how the movie was a joke
----------------------------------------------------


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import math # math.exp を使う場合

def calculate_perplexity_for_sentence(text: str, model, tokenizer, device="cpu"):
    """
    与えられた単一の文に対してパープレキシティを計算します。

    Args:
        text (str): パープレキシティを計算するテキスト文字列。
        model (PreTrainedModel): 事前学習済み言語モデル。
        tokenizer (PreTrainedTokenizer): 対応するトークナイザー。
        device (str): 計算に使用するデバイス ("cpu" または "cuda")。

    Returns:
        tuple: (パープレキシティ, 平均負の対数尤度)
               計算に失敗した場合は (float('inf'), float('inf'))
    """
    model.to(device) # モデルを計算デバイスに移動

    # テキストをトークン化
    # Hugging FaceのモデルでPPLを計算する際は、入力テキスト全体をinput_idsとし、
    # 同じものをlabelsとして渡すのが一般的です。モデル内部で適切にシフトして損失を計算します。
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    # attention_mask = inputs.attention_mask.to(device) # labelsを指定する場合、通常attention_maskは必須ではない

    # 損失（平均NLL）を計算
    with torch.no_grad(): # 勾配計算は不要
        outputs = model(input_ids, labels=input_ids)
        # outputs.loss は、シーケンス全体のトークンごとの平均クロスエントロピー損失（負の対数尤度）
        neg_log_likelihood = outputs.loss

    if neg_log_likelihood is None or torch.isnan(neg_log_likelihood) or torch.isinf(neg_log_likelihood):
        print(f"Warning: Could not compute valid NLL for '{text}'. Assigning high PPL.")
        return float('inf'), float('inf')

    # パープレキシティを計算: PPL = exp(NLL)
    # neg_log_likelihood はスカラテンソルなので .item() でPythonの数値に変換可能
    ppl = torch.exp(neg_log_likelihood).item()
    # または ppl = math.exp(neg_log_likelihood.item())

    return ppl, neg_log_likelihood.item()

if __name__ == "__main__":
    # 使用するモデルを選択 (例: "gpt2", "gpt2-medium", "distilgpt2" など)
    # "gpt2" は比較的小さく、ダウンロードも容易です。
    model_name = "gpt2"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    print(f"\nLoading model and tokenizer for '{model_name}'...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.eval() # 評価モードに設定
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        exit()

    sentences_to_evaluate = [
        "The movie was full of surprises",    # 文法的に正しい (単数)
        "The movies were full of surprises",  # 文法的に正しい (複数)
        "The movie were full of surprises",   # 文法誤り (単数主語 + 複数動詞)
        "The movies was full of surprises"    # 文法誤り (複数主語 + 単数動詞)
    ]

    print(f"\n--- Calculating Perplexity using '{model_name}' ---")
    results = []
    for sentence in sentences_to_evaluate:
        ppl, nll = calculate_perplexity_for_sentence(sentence, model, tokenizer, device)
        results.append({
            "sentence": sentence,
            "nll": nll,
            "perplexity": ppl
        })
        print(f"\nSentence: \"{sentence}\"")
        if math.isinf(ppl):
            print(f"  Negative Log Likelihood (NLL): N/A")
            print(f"  Perplexity (PPL): Infinite / Not Calculable")
        else:
            print(f"  Negative Log Likelihood (NLL): {nll:.4f}")
            print(f"  Perplexity (PPL): {ppl:.4f}")

    print("\n\n--- Observation Summary ---")
    print("Perplexity (PPL) measures how well a probability distribution or probability model predicts a sample.")
    print("A lower PPL indicates the model is less 'surprised' by the sentence, meaning it considers the sentence more probable or natural.")
    print("Generally, we expect:")
    print("  - Grammatically correct sentences to have lower PPL than incorrect ones.")
    print("  - More common or plausible phrasings to have lower PPL.")

    # 結果をPPLでソートして表示
    if any(not math.isinf(res["perplexity"]) for res in results):
        sorted_results = sorted(results, key=lambda x: x["perplexity"])
        print("\nSentences sorted by Perplexity (lower is generally better):")
        for res in sorted_results:
            if math.isinf(res["perplexity"]):
                print(f"  PPL: Infinite - NLL: N/A      - \"{res['sentence']}\"")
            else:
                print(f"  PPL: {res['perplexity']:.2f}     - NLL: {res['nll']:.2f} - \"{res['sentence']}\"")
    else:
        print("\nCould not calculate PPL for any sentence.")

    print("\n--- Expected Outcome based on the input sentences ---")
    print("1. \"The movie was full of surprises\" (Correct Singular): Should have a relatively low PPL.")
    print("2. \"The movies were full of surprises\" (Correct Plural): Should also have a relatively low PPL, possibly similar to the singular correct version.")
    print("3. \"The movie were full of surprises\" (Incorrect - singular noun, plural verb): Should have a higher PPL than (1).")
    print("4. \"The movies was full of surprises\" (Incorrect - plural noun, singular verb): Should have a higher PPL than (2).")
    print("The two grammatically incorrect sentences are expected to show significantly higher PPL values, reflecting the model's 'surprise' or difficulty in predicting such ungrammatical sequences.")

Using device: cuda

Loading model and tokenizer for 'gpt2'...

--- Calculating Perplexity using 'gpt2' ---


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.



Sentence: "The movie was full of surprises"
  Negative Log Likelihood (NLL): 4.5987
  Perplexity (PPL): 99.3545

Sentence: "The movies were full of surprises"
  Negative Log Likelihood (NLL): 4.8401
  Perplexity (PPL): 126.4840

Sentence: "The movie were full of surprises"
  Negative Log Likelihood (NLL): 5.6308
  Perplexity (PPL): 278.8832

Sentence: "The movies was full of surprises"
  Negative Log Likelihood (NLL): 5.6156
  Perplexity (PPL): 274.6661


--- Observation Summary ---
Perplexity (PPL) measures how well a probability distribution or probability model predicts a sample.
A lower PPL indicates the model is less 'surprised' by the sentence, meaning it considers the sentence more probable or natural.
Generally, we expect:
  - Grammatically correct sentences to have lower PPL than incorrect ones.
  - More common or plausible phrasings to have lower PPL.

Sentences sorted by Perplexity (lower is generally better):
  PPL: 99.35     - NLL: 4.60 - "The movie was full of surprises"

## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_chat_response(question: str, model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
    """
    チャットテンプレートを適用してプロンプトを作成し、言語モデルからの応答を生成します。

    Args:
        question (str): ユーザーからの問いかけ。
        model_name (str, optional): 使用する対話型モデルの名前。
                                     デフォルトは "TinyLlama/TinyLlama-1.1B-Chat-v1.0"。
    """
    try:
        print(f"Loading model and tokenizer for '{model_name}'...")
        # 1. モデルとトークナイザーのロード
        # device_map="auto" を使うと、利用可能なGPUに自動でモデルを割り振ります。
        # CPUのみの場合は device_map="cpu" と同様の動作になります。
        # 量子化されたモデルを使う場合は、追加のライブラリや設定が必要なことがあります。
        # 例: model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

        device = model.device # モデルがロードされたデバイスを取得
        print(f"Model loaded on device: {device}")
        model.eval()  # 評価モードに設定

        # 2. 会話履歴の作成
        # システムプロンプトはモデルの振る舞いを指示するのに役立ちます (オプション)
        messages = [
            {"role": "system", "content": "You are a helpful and knowledgeable assistant."},
            {"role": "user", "content": question}
        ]

        # 3. チャットテンプレートの適用
        # tokenize=False: プロンプトを文字列として取得
        # add_generation_prompt=True: モデルが応答を続けるための接尾辞をプロンプトに追加
        # (例: アシスタントの応答開始を示す "<|assistant|>\n" など)
        prompt_string = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        print("\n--- Generated Prompt using Chat Template ---")
        print(f"Template applied by tokenizer for '{model_name}':")
        print(prompt_string)
        print("------------------------------------------")

        # 4. プロンプトをトークン化 (文字列プロンプトを使用する場合)
        # apply_chat_templateでtokenize=Trueにすれば、このステップは不要で直接トークンIDが得られます。
        inputs = tokenizer(prompt_string, return_tensors="pt").to(device)
        input_ids = inputs.input_ids
        # attention_maskも同様に inputs.attention_mask で取得できますが、
        # 単一シーケンスの生成では model.generate が内部で処理してくれることが多いです。

        # 5. 応答の生成
        # pad_token_idをeos_token_idに設定するのは一般的な対処法です。
        # 特にバッチ処理や左パディングが必要なモデルで重要になります。
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        print("\nGenerating response...")
        with torch.no_grad():
            # max_new_tokens: 新しく生成するトークンの最大数
            # temperature: サンプリングの際のランダム性を制御 (低いと決定的、高いと多様)
            # top_k, top_p: サンプリングの候補を絞り込む手法
            generation_output = model.generate(
                input_ids,
                max_new_tokens=60,       # 応答の長さを適度に制限
                do_sample=True,          # サンプリングを有効化
                temperature=0.7,
                top_k=50,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id # パディングトークンIDを指定
            )

        # 生成された応答部分のみをデコード
        # generation_output[0] はプロンプトを含む全体のシーケンス
        # input_ids.shape[-1] はプロンプトのトークン長
        response_ids = generation_output[0][input_ids.shape[-1]:]
        response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

        print("\n--- Generated Response ---")
        print(response_text)
        print("--------------------------")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    user_question = "What do you call a sweet eaten after dinner?"
    generate_chat_response(user_question)

    # (オプション) 他のチャットモデルで試す場合 (モデルサイズやアクセス権に注意)
    # モデルによっては、チャットテンプレートの形式が異なる場合がありますが、
    # tokenizer.apply_chat_template がそれを吸収してくれます。
    # 例:
    # generate_chat_response(user_question, model_name="mistralai/Mistral-7B-Instruct-v0.2")
    # generate_chat_response(user_question, model_name="HuggingFaceH4/zephyr-7b-beta")

Loading model and tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on device: cuda:0

--- Generated Prompt using Chat Template ---
Template applied by tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0':
<|system|>
You are a helpful and knowledgeable assistant.</s>
<|user|>
What do you call a sweet eaten after dinner?</s>
<|assistant|>

------------------------------------------

Generating response...

--- Generated Response ---
A post-dinner treat or dessert.
--------------------------


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_follow_up_chat_response(
    previous_interactions: list,
    follow_up_question: str,
    model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
):
    """
    既存の会話履歴に続けて、新たな質問に対する応答を生成します。

    Args:
        previous_interactions (list): これまでの会話のリスト。各要素は
                                      {"role": "user/assistant/system", "content": "message"} の形式。
        follow_up_question (str): 新たなユーザーからの問いかけ。
        model_name (str, optional): 使用する対話型モデルの名前。
                                     デフォルトは "TinyLlama/TinyLlama-1.1B-Chat-v1.0"。
    """
    try:
        print(f"Loading model and tokenizer for '{model_name}' (if not already cached)...")
        # 1. モデルとトークナイザーのロード
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

        device = model.device
        print(f"Model is on device: {device}")
        model.eval()

        # 2. 会話履歴の更新
        # 新しいユーザーの質問を会話履歴に追加
        current_messages = previous_interactions + [{"role": "user", "content": follow_up_question}]

        # 3. チャットテンプレートの適用
        prompt_string = tokenizer.apply_chat_template(
            current_messages,
            tokenize=False,
            add_generation_prompt=True  # アシスタントの応答開始を促す
        )

        print("\n--- Generated Prompt for the Follow-up Question (including history) ---")
        print(prompt_string)
        print("---------------------------------------------------------------------")

        # 4. プロンプトをトークン化
        inputs = tokenizer(prompt_string, return_tensors="pt").to(device)
        input_ids = inputs.input_ids

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        # 5. 応答の生成
        print("\nGenerating response to the follow-up question...")
        with torch.no_grad():
            generation_output = model.generate(
                input_ids,
                max_new_tokens=80,        # 少し長めの応答も許容できるように調整
                do_sample=True,
                temperature=0.6,          # 少しだけ決定論的な方向に (前回0.7と仮定)
                top_k=50,
                top_p=0.90,               # サンプリング候補を少し絞る
                pad_token_id=tokenizer.eos_token_id
            )

        # 生成された応答部分のみをデコード
        response_ids = generation_output[0][input_ids.shape[-1]:]
        response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

        print("\n--- Generated Response ---")
        print(response_text)
        print("--------------------------")

        # 次の対話のために更新された会話履歴を返す (オプション)
        # updated_messages = current_messages + [{"role": "assistant", "content": response_text.strip()}]
        # return updated_messages


    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    # --- 前回の対話のセットアップ (仮定) ---
    initial_question = "What do you call a sweet eaten after dinner?"
    # 仮のモデル応答。実際には、この応答は最初の質問に対してモデルが生成したものです。
    # ここでは、モデルが "Dessert." と応答したとします。
    # より自然な応答として "That would be dessert." や "It's called a dessert." も考えられます。
    # モデルの挙動によってこの部分は変わります。
    assumed_initial_assistant_response = "A post-dinner treat or dessert."

    # (オプション) システムプロンプト
    system_prompt = {"role": "system", "content": "You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested."}

    # これまでの会話履歴を構築
    # システムプロンプトは最初に入れるのが一般的です
    conversation_history = [
        system_prompt,
        {"role": "user", "content": initial_question},
        {"role": "assistant", "content": assumed_initial_assistant_response}
    ]

    # --- 今回の追加の問いかけ ---
    follow_up_user_question = "Please give me the plural form of the word with its spelling in reverse order."

    # 追加の問いかけに対する応答を生成
    generate_follow_up_chat_response(conversation_history, follow_up_user_question)

Loading model and tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' (if not already cached)...
Model is on device: cuda:0

--- Generated Prompt for the Follow-up Question (including history) ---
<|system|>
You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested.</s>
<|user|>
What do you call a sweet eaten after dinner?</s>
<|assistant|>
A post-dinner treat or dessert.</s>
<|user|>
Please give me the plural form of the word with its spelling in reverse order.</s>
<|assistant|>

---------------------------------------------------------------------

Generating response to the follow-up question...

--- Generated Response ---
The plural form of the word with its spelling in reverse order is "shirts" and "shirts" are the plural form.
--------------------------


## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
